In [2]:
%matplotlib notebook

# general tools
import numpy as np
from numpy import random
import scipy as sp

# tree/simulation
from prosstt import simulation as sim
from prosstt import tree
from prosstt.tree import Tree

# dimensionality reduction and visualization
import anndata as ad
from scanpy.api.tl import diffmap
import matplotlib.pyplot as plt
import matplotlib as mpl

ModuleNotFoundError: No module named 'tree_utils'

In [2]:
random.seed(0)

In [3]:
def this_simulate_H(K, G, a=0.2):
    H = np.reshape(sp.stats.gamma.rvs(a, size=K*G), (K, G))
    return H

In [4]:
t = tree.Tree(modules=8, time=[50]*3, G=200)
sample_time = np.arange(0, t.get_max_time())
gene_scale = np.exp(sp.stats.norm.rvs(loc=0.8, scale=1, size=t.G))

In [5]:
HH = [this_simulate_H(8, 200, a=0.1) for i in range(3)]

In [6]:
Ms = None
while not sim.are_lengths_ok(Ms):
    uMs, Ws, Hs = sim.simulate_branching_data(t, HH=HH)
    Ms = [np.zeros((t.time[i], t.G)) for i in range(t.branches)]
    for i in range(t.branches):
        Ms[i] = np.exp(uMs[i]) * gene_scale

t.add_genes(Ms)

/home/npapado/miniconda2/envs/py36/lib/python3.6/site-packages/scipy/stats/stats.py:3003: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


In [7]:
alpha = np.exp(random.normal(loc=np.log(0.2), scale=np.log(1.5), size=t.G))
beta = np.exp(random.normal(loc=np.log(1), scale=np.log(1.5), size=t.G)) + 1

X, labs, brns, scalings = sim.sample_data_balanced(1, t.G, t, sample_time, alpha, beta)

# normalize gene expression by library size
X = (X.transpose() / scalings).transpose()

 |███████████████████████████████████████████████████████████████████████████████████████████████████-| 99.3% 

In [1]:
data = ad.AnnData(X)
diffmap(adata=data)
dm = data.obsm["X_diffmap"]

fig, ax = plt.subplots(ncols=2)
fig.set_size_inches(w=9, h=4)
ax[0].scatter(dm[:, 0], dm[:, 1], c=brns, cmap="Set3")
ax[0].set_title("branches")
# mpl.rcParams['image.cmap'] = 'viridis'
ax[1].scatter(dm[:, 0], dm[:, 1], c=labs, cmap="viridis")
ax[1].set_title("pseudotime")

NameError: name 'ad' is not defined